In [1]:
import os
import numpy as np
import pandas as pd
import glob
from itertools import compress
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
mx_names =["Gene-level copy number", "Gene-level threshold", "Region-level copy number", "Region-level threshold"]
sumfeatures = ["# features", "Mean", "Min", "25%", "Median", "75%", "Max", "% positive entries", "% negative entries", "% zero entries"]

In [3]:
def mx_sumstats(mx):
    sumstat_dict = {}
    sumfeatures = ["# features", "Mean", "Min", "25%", "Median", "75%", "Max", "% positive entries", "% negative entries", "% zero entries"]
    for sumfeature in sumfeatures:
        sumstat_dict[sumfeatures[0]] = len(mx.columns) # number of features 
        sumstat_dict[sumfeatures[1]] = np.mean(mx.to_numpy().ravel()) # mean of all entries 
        sumstat_dict[sumfeatures[2]] = np.quantile(mx.to_numpy().ravel(), 0)
        sumstat_dict[sumfeatures[3]] = np.quantile(mx.to_numpy().ravel(), 0.25)
        sumstat_dict[sumfeatures[4]] = np.quantile(mx.to_numpy().ravel(), 0.5)
        sumstat_dict[sumfeatures[5]] = np.quantile(mx.to_numpy().ravel(), 0.75)
        sumstat_dict[sumfeatures[6]] = np.quantile(mx.to_numpy().ravel(), 1)
        sumstat_dict[sumfeatures[7]] = 100*np.sum(mx.to_numpy().ravel()>0)/len(mx.to_numpy().ravel())
        sumstat_dict[sumfeatures[8]] = 100*np.sum(mx.to_numpy().ravel()<0)/len(mx.to_numpy().ravel())
        sumstat_dict[sumfeatures[9]] = 100*np.sum(mx.to_numpy().ravel()==0)/len(mx.to_numpy().ravel())
    return sumstat_dict

In [21]:
def applyThres(table, thres=0.05):
    # table: Pandas DataFrame
    # thres: float
    bool_col = [] # list of boolean indicating the columns to keep at True
    for colname in table.columns:
        ct = 0
        for i in range(len(table[colname])):
            if table[colname][i] != 0:
                ct += 1
            else:
                continue
        rate = ct/len(table[colname])
        if ((rate < thres) | (rate > (1-thres))): # change this so that we only eliminate features that are rare?? (because common can add value?)
            bool_col.append(False)
        else:
            bool_col.append(True)    
    return table.iloc[:,bool_col]

def applyThres_featuresize(table, thres=[0.05, 0.10, 0.20, 0.25]):
    # fs_dict is a dictionary summarizing how feature size is affected by freq-based reduction 
    fs_dict = {}
    fs_dict["Original"] = table.shape[1]
    fs_dict["0.05"] = applyThres(table, thres=0.05).shape[1]
    fs_dict["0.10"] = applyThres(table, thres=0.10).shape[1]
    fs_dict["0.20"] = applyThres(table, thres=0.20).shape[1]
    fs_dict["0.25"] = applyThres(table, thres=0.25).shape[1]
    return fs_dict


# Overall Distribution of GISTIC2.0 outputs with conf=0.75

In [5]:
gene_copy = pd.read_csv("../../gistic2_out_conf75/gene_copy_conf75.csv", header=0, index_col=0)
gene_thres = pd.read_csv("../../gistic2_out_conf75/gene_thres_conf75.csv", header=0, index_col=0)
reg_copy = pd.read_csv("../../gistic2_out_conf75/reg_copy_conf75.csv", header=0, index_col=0).T
reg_thres = pd.read_csv("../../gistic2_out_conf75/reg_thres_conf75.csv", header=0, index_col=0).T

In [6]:
mx_dicts = {}

for i in range(len([gene_copy, gene_thres, reg_copy, reg_thres])):
    mx_dicts[mx_names[i]] = [gene_copy, gene_thres, reg_copy, reg_thres][i]

In [7]:
sum_table = pd.DataFrame(None, index=mx_names, columns=sumfeatures)

for mxkey in mx_dicts.keys():
    sumstatdict = mx_sumstats(mx_dicts[mxkey])
    for sumkey in sumstatdict.keys():
        sum_table.loc[mxkey][sumkey] = sumstatdict[sumkey]

In [8]:
sum_table.T

,Gene-level copy number,Gene-level threshold,Region-level copy number,Region-level threshold
# features,25988,25988,638,638
Mean,0.0193007,0.135262,-0.0171783,-0.0171783
Min,-1.293,-2,-1.2929,-1.2929
25%,-0.045,0,-0.0981028,-0.0981028
Median,0.006,0,5.1987e-05,5.1987e-05
75%,0.093,0,0.10412,0.10412
Max,3.657,2,3.6569,3.6569
% positive entries,53.4197,23.9989,50.0476,50.0476
% negative entries,43.6112,16.088,48.0856,48.0856
% zero entries,2.96913,59.9131,1.86684,1.86684


In [25]:
thresholds = ["Original", "0.05", "0.10", "0.20", "0.25"]
fs_all = pd.DataFrame(None, index=thresholds, columns=mx_dicts.keys())

# How many features will the matrices have after frequency-based reduction? 
for mxkey in mx_dicts.keys():
    fsdict = applyThres_featuresize(mx_dicts[mxkey])
    for thres in thresholds:
        fs_all.loc[thres][mxkey] = fsdict[thres]
display(fs_all)

,Gene-level copy number,Gene-level threshold,Region-level copy number,Region-level threshold
Original,25988,25988,638,638
0.05,5569,25786,72,72
0.10,638,25119,2,2
0.20,0,18756,0,0
0.25,0,14983,0,0


# Overall distribution of GISTIC2.0 outputs with conf=0.90

In [26]:
gene_copy = pd.read_csv("../../gistic2_out_conf90/gene_copy_conf90.csv", header=0, index_col=0)
gene_thres = pd.read_csv("../../gistic2_out_conf90/gene_thres_conf90.csv", header=0, index_col=0)
reg_copy = pd.read_csv("../../gistic2_out_conf90/reg_copy_conf90.csv", header=0, index_col=0).T
reg_thres = pd.read_csv("../../gistic2_out_conf90/reg_thres_conf90.csv", header=0, index_col=0).T

In [27]:
mx_dicts = {}

for i in range(len([gene_copy, gene_thres, reg_copy, reg_thres])):
    mx_dicts[mx_names[i]] = [gene_copy, gene_thres, reg_copy, reg_thres][i]

In [28]:
sum_table = pd.DataFrame(None, index=mx_names, columns=sumfeatures)

for mxkey in mx_dicts.keys():
    sumstatdict = mx_sumstats(mx_dicts[mxkey])
    for sumkey in sumstatdict.keys():
        sum_table.loc[mxkey][sumkey] = sumstatdict[sumkey]

In [29]:
sum_table.T

,Gene-level copy number,Gene-level threshold,Region-level copy number,Region-level threshold
# features,25988,25988,638,638
Mean,0.0193007,0.135262,-0.0171783,0.373438
Min,-1.293,-2,-1.2929,0
25%,-0.045,0,-0.0981028,0
Median,0.006,0,5.1987e-05,0
75%,0.093,0,0.10412,1
Max,3.657,2,3.6569,2
% positive entries,53.4197,23.9989,50.0476,35.0933
% negative entries,43.6112,16.088,48.0856,0
% zero entries,2.96913,59.9131,1.86684,64.9067


In [30]:
# How many features will the matrices have after frequency-based reduction? 
thresholds = ["Original", "0.05", "0.10", "0.20", "0.25"]
fs_all = pd.DataFrame(None, index=thresholds, columns=mx_dicts.keys())

# How many features will the matrices have after frequency-based reduction? 
for mxkey in mx_dicts.keys():
    fsdict = applyThres_featuresize(mx_dicts[mxkey])
    for thres in thresholds:
        fs_all.loc[thres][mxkey] = fsdict[thres]
display(fs_all)

,Gene-level copy number,Gene-level threshold,Region-level copy number,Region-level threshold
Original,25988,25988,638,638
0.05,5569,25786,72,637
0.10,638,25119,2,625
0.20,0,18756,0,472
0.25,0,14983,0,357


# Overall distributions for in-house feature matrices 

In [31]:
mx_names = ["refGene-based feature mx", "GATK-based feature mx"]

In [32]:
mx_dicts = {}

mx_dicts[mx_names[0]] = pd.read_csv("../../inhouse_features_output/refGene_cnv_gene_matrix.csv", header=0, index_col=0)
mx_dicts[mx_names[1]] = pd.read_csv("../../inhouse_features_output/gatk_cnv_gene_matrix.csv", header=0, index_col=0)

In [33]:
sum_table = pd.DataFrame(None, index=mx_names, columns=sumfeatures)

for mxkey in mx_dicts.keys():
    sumstatdict = mx_sumstats(mx_dicts[mxkey])
    for sumkey in sumstatdict.keys():
        sum_table.loc[mxkey][sumkey] = sumstatdict[sumkey]

In [34]:
sum_table.T

,refGene-based feature mx,GATK-based feature mx
# features,28046,55735
Mean,-0.0668399,-0.0596054
Min,-31.5209,-31.5209
25%,0,0
Median,0,0
75%,0,0
Max,6.50117,6.50117
% positive entries,13.6156,11.5889
% negative entries,10.97,11.195
% zero entries,75.4143,77.2161


In [37]:
# How many features will the matrices have after frequency-based reduction? 
thresholds = ["Original", "0.05", "0.10", "0.20", "0.25"]
fs_all = pd.DataFrame(None, index=thresholds, columns=mx_dicts.keys())

# How many features will the matrices have after frequency-based reduction? 
for mxkey in mx_dicts.keys():
    fsdict = applyThres_featuresize(mx_dicts[mxkey])
    for thres in thresholds:
        fs_all.loc[thres][mxkey] = fsdict[thres]
display(fs_all)

,refGene-based feature mx,GATK-based feature mx
Original,28046,55735
0.05,23887,46151
0.10,18829,35446
0.20,12800,22846
0.25,10566,18569


## Inspect which features are causing negative skew in in-house features

In [49]:
# in refGene matrix
# how many features have values <-5.0? how many have < -10.0? <-20.0? and <-30.0?? 
print("Number of features that have values < -5 in refGene: {}".format(
    np.sum([np.any(mx_dicts[mx_names[0]].iloc[:,x] < -5) for x in range(mx_dicts[mx_names[0]].shape[0])])
))
print("Number of features that have values < -10 in refGene: {}".format(
    np.sum([np.any(mx_dicts[mx_names[0]].iloc[:,x] < -10) for x in range(mx_dicts[mx_names[0]].shape[0])])
))
print("Number of features that have values < -20 in refGene: {}".format(
    np.sum([np.any(mx_dicts[mx_names[0]].iloc[:,x] < -20) for x in range(mx_dicts[mx_names[0]].shape[0])])
))
print("Number of features that have values < -30 in refGene: {}".format(
    np.sum([np.any(mx_dicts[mx_names[0]].iloc[:,x] < -30) for x in range(mx_dicts[mx_names[0]].shape[0])])
))

Number of features that have values < -5 in refGene: 23
Number of features that have values < -10 in refGene: 21
Number of features that have values < -20 in refGene: 8
Number of features that have values < -30 in refGene: 2


In [57]:
print("Gene features that had highly negative values (< -20) in refGene:")
print(list(
    compress(
        list(mx_dicts[mx_names[0]].columns), 
        [np.any(mx_dicts[mx_names[0]].iloc[:,x] < -20) for x in range(mx_dicts[mx_names[0]].shape[0])]
    )
))

Gene features that had highly negative values (< -20) in refGene:
['LOC400627', 'KHDRBS3', 'FAM66D', 'URAHP', 'PRAMEF6', 'TMEM220-AS1', 'FAM157C', 'GRK4']


In [58]:
# in GATK matrix
# how many features have values <-5.0? how many have < -10.0? <-20.0? and <-30.0?? 
print("Number of features that have values < -5 in GATK: {}".format(
    np.sum([np.any(mx_dicts[mx_names[1]].iloc[:,x] < -5) for x in range(mx_dicts[mx_names[0]].shape[0])])
))
print("Number of features that have values < -10 in GATK: {}".format(
    np.sum([np.any(mx_dicts[mx_names[1]].iloc[:,x] < -10) for x in range(mx_dicts[mx_names[0]].shape[0])])
))
print("Number of features that have values < -20 in GATK: {}".format(
    np.sum([np.any(mx_dicts[mx_names[1]].iloc[:,x] < -20) for x in range(mx_dicts[mx_names[0]].shape[0])])
))
print("Number of features that have values < -30 in GATK: {}".format(
    np.sum([np.any(mx_dicts[mx_names[1]].iloc[:,x] < -30) for x in range(mx_dicts[mx_names[0]].shape[0])])
))

Number of features that have values < -5 in GATK: 17
Number of features that have values < -10 in GATK: 15
Number of features that have values < -20 in GATK: 7
Number of features that have values < -30 in GATK: 1


In [60]:
print("Gene features that had highly negative values (< -20) in GATK:")
print(list(
    compress(
        list(mx_dicts[mx_names[1]].columns), 
        [np.any(mx_dicts[mx_names[1]].iloc[:,x] < -20) for x in range(mx_dicts[mx_names[1]].shape[0])]
    )
))

Gene features that had highly negative values (< -20) in GATK:
['Y_RNA', 'FAM157C', 'TRIM64C', 'GAS7', 'MIR3179-1', 'VSTM1', 'TMEM220-AS1']


## How many features in GATK have patterns "AC" "AR" "CR" etc. ??? 

In [62]:
ac_list = []
gene_list = list(mx_dicts[mx_names[1]].columns)

for i in range(len(gene_list)):
    if re.search(r"\bAC[0-9]", gene_list[i]) != None:
        ac_list.append(gene_list[i])

print("Number of genes with pattern AC<num>: ", len(ac_list))
print("Sample: ", ac_list[:5])

Number of genes with pattern AC<num>:  12791
Sample:  ['AC004801.7', 'AC137056.2', 'AC012414.2', 'AC010551.3', 'AC024361.2']


In [66]:
al_list = []
for i in range(len(gene_list)):
    if re.search(r"\bAL[0-9]", gene_list[i]) != None:
        al_list.append(gene_list[i])

print("Number of genes with pattern AL<num>: ", len(al_list))
print("Sample: ", al_list[:5])

Number of genes with pattern AL<num>:  4744
Sample:  ['AL136164.2', 'AL133367.1', 'AL161773.1', 'AL132712.2', 'AL161722.1']


In [67]:
ap_list = []
for i in range(len(gene_list)):
    if re.search(r"\bAP[0-9]", gene_list[i]) != None:
        ap_list.append(gene_list[i])

print("Number of genes with pattern AP<num>: ", len(ap_list))
print("Sample: ", ap_list[:5])

Number of genes with pattern AL<num>:  1206
Sample:  ['AP002748.2', 'AP003096.1', 'AP001462.1', 'AP003716.1', 'AP003774.1']


In [68]:
cr_list = []
for i in range(len(gene_list)):
    if re.search(r"\bCR[0-9]", gene_list[i]) != None:
        cr_list.append(gene_list[i])

print("Number of genes with pattern CR<num>: ", len(cr_list))
print("Sample: ", cr_list[:5])

Number of genes with pattern AL<num>:  44
Sample:  ['CR383656.4', 'CR383656.9', 'CR559946.1', 'CR383656.1', 'CR383656.5']


In [63]:
mir_list = []
for i in range(len(gene_list)):
    if re.search(r"\bMIR[0-9]", gene_list[i]) != None:
        mir_list.append(gene_list[i])

print("Number of genes with pattern MIR<num>: ", len(mir_list))
print("Sample: ", mir_list[:5])

Number of genes with pattern MIR<num>:  1901
Sample:  ['MIR4330', 'MIR4507', 'MIR526A1', 'MIR3179-1', 'MIR4648']


In [65]:
linc_list = []
for i in range(len(gene_list)):
    if re.search(r"\bLINC[0-9]", gene_list[i]) != None:
        linc_list.append(gene_list[i])

print("Number of genes with pattern LINC<num>: ", len(linc_list))
print("Sample: ", linc_list[:5])

Number of genes with pattern LINC<num>:  1849
Sample:  ['LINC00565', 'LINC01864', 'LINC02369', 'LINC00893', 'LINC01704']
